In [1]:
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from joblib import Parallel, delayed

import lightgbm as lgbm
import optuna
from optuna.samplers import TPESampler
#import optuna.integration.lightgbm as lgbm
from sklearn.model_selection import GroupKFold, KFold, StratifiedKFold

pd.set_option('max_columns', 150)

In [2]:
def rmspe(preds, ys): return (((preds - ys) / ys) ** 2).mean() ** 0.5

# Function to early stop with root mean squared percentage error
def feval_rmspe(y_pred, lgb_train):
    y_true = lgb_train.get_label()
    return 'RMSPE', rmspe(y_pred, y_true), False

In [4]:
df = pd.read_csv('train.csv')
df

,stock_id,time_id,target,row_id,wap1_std,wap1_sum,wap1_amax,wap1_amin,wap2_std,wap2_sum,wap2_amax,wap2_amin,wap3_std,wap3_sum,wap3_amax,wap3_amin,wap4_std,wap4_sum,wap4_amax,wap4_amin,log_return1_realized_volatility,log_return1_std,log_return1_sum,log_return2_realized_volatility,log_return2_std,log_return2_sum,log_return3_realized_volatility,log_return3_std,log_return3_sum,log_return4_realized_volatility,log_return4_std,log_return4_sum,wap_balance_sum,wap_balance_std,price_spread1_sum,price_spread1_std,price_spread2_sum,price_spread2_std,bid_spread_sum,bid_spread_std,ask_spread_sum,ask_spread_std,bid_ask_spread_sum,bid_ask_spread_std,total_volume_sum,total_volume_std,volume_imbalance_sum,volume_imbalance_std,log_return1_realized_volatility_150,log_return2_realized_volatility_150,log_return3_realized_volatility_150,log_return4_realized_volatility_150,log_return1_realized_volatility_300,log_return2_realized_volatility_300,log_return3_realized_volatility_300,log_return4_realized_volatility_300,log_return1_realized_volatility_450,log_return2_realized_volatility_450,log_return3_realized_volatility_450,log_return4_realized_volatility_450,trade_log_return_realized_volatility,trade_log_return_std,trade_log_return_sum,trade_seconds_in_bucket_count_unique,trade_size_sum,trade_size_std,trade_order_count_sum,trade_order_count_std,trade_amount_sum,trade_amount_std,trade_log_return_realized_volatility_150,trade_seconds_in_bucket_count_unique_150,trade_size_sum_150,trade_order_count_sum_150,trade_log_return_realized_volatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,trade_order_count_sum_300,trade_log_return_realized_volatility_450,trade_seconds_in_bucket_count_unique_450,trade_size_sum_450,trade_order_count_sum_450
0,0,5,0.004136,0-5,0.000693,303.125061,1.004920,1.001434,0.000781,303.105539,1.005124,1.001390,0.000637,303.134863,1.004795,1.001692,0.000652,303.146936,1.004976,1.002334,0.004499,0.000260,0.002292,0.006999,0.000404,0.002325,0.005466,0.000316,0.000855,0.006119,0.000353,0.000770,0.117051,0.000295,0.257255,0.000211,0.355451,0.000213,0.053006,0.000162,-0.045557,0.000126,0.098563,0.000222,97696.0,138.101214,40738.0,107.260583,0.003796,0.006087,0.004942,0.005564,0.002953,0.004863,0.003873,0.004492,0.001721,0.004114,0.002599,0.003368,0.002006,0.000324,0.001341,40.0,3179.0,118.375107,110.0,2.467741,3190.139181,118.778412,0.001701,30.0,2069.0,73.0,0.001308,21.0,1587.0,54.0,0.001060,14.0,1042.0,37.0
1,0,11,0.001445,0-11,0.000262,200.047768,1.000834,0.999700,0.000272,200.041171,1.001067,0.999575,0.000298,200.035611,1.000840,0.999499,0.000278,200.040851,1.000882,0.999558,0.001204,0.000086,0.000360,0.002476,0.000176,0.000801,0.001782,0.000127,0.001045,0.002320,0.000165,0.000253,0.042312,0.000155,0.078836,0.000157,0.134182,0.000200,0.028358,0.000148,-0.027001,0.000065,0.055359,0.000158,82290.0,172.263581,28410.0,102.139758,0.001058,0.002262,0.001509,0.002052,0.000981,0.002009,0.001308,0.001637,0.000918,0.001883,0.001169,0.001438,0.000901,0.000168,0.000803,30.0,1289.0,77.815203,57.0,1.446756,1289.353432,77.834931,0.000813,24.0,1173.0,49.0,0.000587,16.0,900.0,36.0,0.000501,10.0,828.0,22.0
2,0,16,0.002168,0-16,0.000864,187.913849,1.000878,0.997224,0.000862,187.939824,1.000876,0.996896,0.000670,187.923063,1.000498,0.997463,0.000779,187.897375,1.000918,0.996791,0.002369,0.000173,-0.002074,0.004801,0.000352,-0.001493,0.002624,0.000192,-0.002335,0.004684,0.000343,-0.002868,0.062228,0.000246,0.136330,0.000164,0.210563,0.000295,0.036955,0.000170,-0.037243,0.000171,0.074198,0.000220,78274.0,138.433034,26586.0,108.891243,0.002138,0.004019,0.002368,0.003845,0.001295,0.003196,0.001815,0.002354,0.001158,0.002972,0.001703,0.001947,0.001961,0.000394,-0.002540,25.0,2161.0,113.587000,68.0,2.300725,2158.608928,113.450182,0.001621,20.0,2010.0,59.0,0.001137,12.0,1189.0,38.0,0.001048,9.0,1085.0,33.0
3,0,31,0.002195,0-31,0.000757,119.859781,1.000412,0.997447,0.000656,119.835941,1.000116,0.997430,0.000606,119.870163,1.000172,0.998055,0.000733

In [5]:
columns = df.keys().tolist()
[columns.pop(columns.index(c)) for c in  ['time_id', 'row_id', 'target']]

y_data = df['target']
x_data = df[columns]
x_data

,stock_id,wap1_std,wap1_sum,wap1_amax,wap1_amin,wap2_std,wap2_sum,wap2_amax,wap2_amin,wap3_std,wap3_sum,wap3_amax,wap3_amin,wap4_std,wap4_sum,wap4_amax,wap4_amin,log_return1_realized_volatility,log_return1_std,log_return1_sum,log_return2_realized_volatility,log_return2_std,log_return2_sum,log_return3_realized_volatility,log_return3_std,log_return3_sum,log_return4_realized_volatility,log_return4_std,log_return4_sum,wap_balance_sum,wap_balance_std,price_spread1_sum,price_spread1_std,price_spread2_sum,price_spread2_std,bid_spread_sum,bid_spread_std,ask_spread_sum,ask_spread_std,bid_ask_spread_sum,bid_ask_spread_std,total_volume_sum,total_volume_std,volume_imbalance_sum,volume_imbalance_std,log_return1_realized_volatility_150,log_return2_realized_volatility_150,log_return3_realized_volatility_150,log_return4_realized_volatility_150,log_return1_realized_volatility_300,log_return2_realized_volatility_300,log_return3_realized_volatility_300,log_return4_realized_volatility_300,log_return1_realized_volatility_450,log_return2_realized_volatility_450,log_return3_realized_volatility_450,log_return4_realized_volatility_450,trade_log_return_realized_volatility,trade_log_return_std,trade_log_return_sum,trade_seconds_in_bucket_count_unique,trade_size_sum,trade_size_std,trade_order_count_sum,trade_order_count_std,trade_amount_sum,trade_amount_std,trade_log_return_realized_volatility_150,trade_seconds_in_bucket_count_unique_150,trade_size_sum_150,trade_order_count_sum_150,trade_log_return_realized_volatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,trade_order_count_sum_300,trade_log_return_realized_volatility_450,trade_seconds_in_bucket_count_unique_450,trade_size_sum_450,trade_order_count_sum_450
0,0,0.000693,303.125061,1.004920,1.001434,0.000781,303.105539,1.005124,1.001390,0.000637,303.134863,1.004795,1.001692,0.000652,303.146936,1.004976,1.002334,0.004499,0.000260,0.002292,0.006999,0.000404,0.002325,0.005466,0.000316,0.000855,0.006119,0.000353,0.000770,0.117051,0.000295,0.257255,0.000211,0.355451,0.000213,0.053006,0.000162,-0.045557,0.000126,0.098563,0.000222,97696.0,138.101214,40738.0,107.260583,0.003796,0.006087,0.004942,0.005564,0.002953,0.004863,0.003873,0.004492,0.001721,0.004114,0.002599,0.003368,0.002006,0.000324,0.001341,40.0,3179.0,118.375107,110.0,2.467741,3190.139181,118.778412,0.001701,30.0,2069.0,73.0,0.001308,21.0,1587.0,54.0,0.001060,14.0,1042.0,37.0
1,0,0.000262,200.047768,1.000834,0.999700,0.000272,200.041171,1.001067,0.999575,0.000298,200.035611,1.000840,0.999499,0.000278,200.040851,1.000882,0.999558,0.001204,0.000086,0.000360,0.002476,0.000176,0.000801,0.001782,0.000127,0.001045,0.002320,0.000165,0.000253,0.042312,0.000155,0.078836,0.000157,0.134182,0.000200,0.028358,0.000148,-0.027001,0.000065,0.055359,0.000158,82290.0,172.263581,28410.0,102.139758,0.001058,0.002262,0.001509,0.002052,0.000981,0.002009,0.001308,0.001637,0.000918,0.001883,0.001169,0.001438,0.000901,0.000168,0.000803,30.0,1289.0,77.815203,57.0,1.446756,1289.353432,77.834931,0.000813,24.0,1173.0,49.0,0.000587,16.0,900.0,36.0,0.000501,10.0,828.0,22.0
2,0,0.000864,187.913849,1.000878,0.997224,0.000862,187.939824,1.000876,0.996896,0.000670,187.923063,1.000498,0.997463,0.000779,187.897375,1.000918,0.996791,0.002369,0.000173,-0.002074,0.004801,0.000352,-0.001493,0.002624,0.000192,-0.002335,0.004684,0.000343,-0.002868,0.062228,0.000246,0.136330,0.000164,0.210563,0.000295,0.036955,0.000170,-0.037243,0.000171,0.074198,0.000220,78274.0,138.433034,26586.0,108.891243,0.002138,0.004019,0.002368,0.003845,0.001295,0.003196,0.001815,0.002354,0.001158,0.002972,0.001703,0.001947,0.001961,0.000394,-0.002540,25.0,2161.0,113.587000,68.0,2.300725,2158.608928,113.450182,0.001621,20.0,2010.0,59.0,0.001137,12.0,1189.0,38.0,0.001048,9.0,1085.0,33.0
3,0,0.000757,119.859781,1.000412,0.997447,0.000656,119.835941,1.000116,0.997430,0.000606,119.870163,1.000172,0.998055,0.000733,119.884240,1.000569,0.998007,0.002574,0.000236,-0.002828,0.003637,0.000334,-0.002053,0.

In [13]:
def fit_lgbm_sk(x_train,
                y_train,
                x_valid,
                y_valid,
                params,
                verbose=100):
    oof_pred = np.zeros(len(y_valid), dtype=np.float32)
    model = lgbm.LGBMRegressor(**params)
    model.fit(x_train,
              y_train,
              eval_set=[(x_valid, y_valid)],
              verbose=verbose)
    oof_pred = model.predict(x_valid)
    score = rmspe(oof_pred, y_valid)
    return oof_pred, model, score


def fit_lgbm(x_train,
             y_train,
             x_valid,
             y_valid,
             params,
             verbose_eval=100,
             feval=None):

    weights = (1. / (np.square(y_train) + 1e-8))
    train_dataset = lgbm.Dataset(x_train, y_train, weight=weights)
    val_dataset = lgbm.Dataset(x_valid, y_valid)

    oof_pred = np.zeros(len(y_valid), dtype=np.float32)
    model = lgbm.train(params=params,
                       train_set=train_dataset,
                       valid_sets=[train_dataset, val_dataset],
                       verbose_eval=verbose_eval,
                       feval=feval)

    oof_pred = model.predict(x_valid)
    score = rmspe(oof_pred, y_valid)
    return oof_pred, model, score


def objective(trial):
    params = {
        'objective': trial.suggest_categorical('objective', ['rmse', 'l2']),
        # 'n_jobs': 6,
        'verbose': -1,
        'feature_pre_filter': False,
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 2, 512),
        'feature_fraction': trial.suggest_float('feature_fraction', 1e-8, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 1e-8, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'num_iterations': 5000,
        'early_stopping_round': 50,
    }

    ys = []
    preds = []
    # kfold
    kfd = StratifiedKFold(5)
    for trn_idx, val_idx in kfd.split(df, y=df.stock_id, groups=df.time_id):
        x_trn = x_data.iloc[trn_idx]
        y_trn = y_data.iloc[trn_idx]

        x_val = x_data.iloc[val_idx]
        y_val = y_data.iloc[val_idx]

        oof_pred, model, score = fit_lgbm(x_trn, 
                                          y_trn, 
                                          x_val, 
                                          y_val, 
                                          params, 
                                          verbose_eval=-1, 
                                          feval=feval_rmspe)
        ys.append(y_val)
        preds.append(oof_pred)
    ys = np.concatenate(ys, axis=0)
    preds = np.concatenate(preds, axis=0)
    return rmspe(preds, ys)

In [14]:
sampler = TPESampler(seed=123)
study = optuna.create_study(direction='minimize', sampler=sampler)
study.optimize(objective, n_trials=30)

[I 2021-09-27 12:47:54,228] A new study created in memory with name: no-name-21de6660-2490-43b5-bb73-8fa6ee169354
C:\Users\user\anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\user\anaconda3\lib\site-packages\lightgbm\engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[231]	training's rmse: 0.000490965	training's RMSPE: 0.226668	valid_1's rmse: 0.00142672	valid_1's RMSPE: 0.236009
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[187]	training's rmse: 0.000495952	training's RMSPE: 0.228043	valid_1's rmse: 0.0013234	valid_1's RMSPE: 0.233121
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[353]	training's rmse: 0.000486404	training's RMSPE: 0.224199	valid_1's rmse: 0.00134786	valid_1's RMSPE: 0.23177
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[434]	training's rmse: 0.000477811	training's RMSPE: 0.22136	valid_1's rmse: 0.0014963	valid_1's RMSPE: 0.243705
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[93]	training's rmse: 0.000507309	training's RMSPE: 0.232895	val

[I 2021-09-27 12:49:09,754] Trial 0 finished with value: 0.23664734826282754 and parameters: {'objective': 'rmse', 'learning_rate': 0.07158257673179949, 'lambda_l1': 7.181883164075261e-05, 'lambda_l2': 0.01651823708592784, 'num_leaves': 19, 'feature_fraction': 0.7194689725908734, 'bagging_fraction': 0.42310646589339634, 'bagging_freq': 7, 'min_child_samples': 52}. Best is trial 0 with value: 0.23664734826282754.
C:\Users\user\anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\user\anaconda3\lib\site-packages\lightgbm\engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[379]	training's l2: 1.94219e-07	training's RMSPE: 0.203415	valid_1's l2: 1.91363e-06	valid_1's RMSPE: 0.235786
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[294]	training's l2: 2.0566e-07	training's RMSPE: 0.208475	valid_1's l2: 1.6711e-06	valid_1's RMSPE: 0.233429
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[464]	training's l2: 1.87033e-07	training's RMSPE: 0.19929	valid_1's l2: 1.72632e-06	valid_1's RMSPE: 0.230251
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[275]	training's l2: 2.02704e-07	training's RMSPE: 0.208536	valid_1's l2: 2.20449e-06	valid_1's RMSPE: 0.245363
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[223]	training's l2: 2.1753e-07	training's RMSPE: 0.213981	valid_1's l2: 1.

[I 2021-09-27 12:51:32,513] Trial 1 finished with value: 0.23655469952383856 and parameters: {'objective': 'l2', 'learning_rate': 0.04070073810945915, 'lambda_l1': 0.004424386760115344, 'lambda_l2': 8.267192313566323e-06, 'num_leaves': 336, 'feature_fraction': 0.05967790601278938, 'bagging_fraction': 0.3980442613499888, 'bagging_freq': 5, 'min_child_samples': 54}. Best is trial 1 with value: 0.23655469952383856.
C:\Users\user\anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\user\anaconda3\lib\site-packages\lightgbm\engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


...

[I 2021-09-27 15:00:38,836] Trial 27 finished with value: 0.23365151716426594 and parameters: {'objective': 'rmse', 'learning_rate': 0.01158776389346235, 'lambda_l1': 7.660156871108371e-05, 'lambda_l2': 0.005113447265461464, 'num_leaves': 373, 'feature_fraction': 0.5973119340163503, 'bagging_fraction': 0.1535164267934673, 'bagging_freq': 5, 'min_child_samples': 42}. Best is trial 26 with value: 0.23243422762975208.
C:\Users\user\anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\user\anaconda3\lib\site-packages\lightgbm\engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1854]	training's l2: 2.11234e-07	training's RMSPE: 0.212233	valid_1's l2: 1.93458e-06	valid_1's RMSPE: 0.232104
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1813]	training's l2: 2.13668e-07	training's RMSPE: 0.212583	valid_1's l2: 1.66254e-06	valid_1's RMSPE: 0.229157
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[2583]	training's l2: 2.01606e-07	training's RMSPE: 0.206966	valid_1's l2: 1.72713e-06	valid_1's RMSPE: 0.22704
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1986]	training's l2: 2.05084e-07	training's RMSPE: 0.209827	valid_1's l2: 2.18331e-06	valid_1's RMSPE: 0.240449
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1159]	training's l2: 2.29281e-07	training's RMSPE: 0.21989	valid_1's

[I 2021-09-27 15:12:44,842] Trial 28 finished with value: 0.23253356105177564 and parameters: {'objective': 'l2', 'learning_rate': 0.0038763112882897926, 'lambda_l1': 1.0675070173763283e-05, 'lambda_l2': 8.556977261914438e-05, 'num_leaves': 261, 'feature_fraction': 0.3814386592346447, 'bagging_fraction': 0.35485276037217883, 'bagging_freq': 4, 'min_child_samples': 93}. Best is trial 26 with value: 0.23243422762975208.
C:\Users\user\anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\user\anaconda3\lib\site-packages\lightgbm\engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[3064]	training's rmse: 0.000460345	training's RMSPE: 0.212591	valid_1's rmse: 0.00139273	valid_1's RMSPE: 0.232023
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[3262]	training's rmse: 0.000460004	training's RMSPE: 0.211561	valid_1's rmse: 0.00128795	valid_1's RMSPE: 0.229278
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[4176]	training's rmse: 0.000450309	training's RMSPE: 0.207579	valid_1's rmse: 0.00131671	valid_1's RMSPE: 0.227022
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[3170]	training's rmse: 0.000454852	training's RMSPE: 0.210779	valid_1's rmse: 0.00148029	valid_1's RMSPE: 0.241247
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[2249]	training's rmse: 0.00047462	training's RMSPE: 0.2

[I 2021-09-27 15:31:27,917] Trial 29 finished with value: 0.2326773609534869 and parameters: {'objective': 'rmse', 'learning_rate': 0.002100402716226725, 'lambda_l1': 2.2237989411523872e-05, 'lambda_l2': 0.00015481852772935217, 'num_leaves': 294, 'feature_fraction': 0.35915290322889754, 'bagging_fraction': 0.3680107036518758, 'bagging_freq': 4, 'min_child_samples': 92}. Best is trial 26 with value: 0.23243422762975208.


In [22]:
study.best_params, study.best_value

({'objective': 'l2',
  'learning_rate': 0.016878642221576215,
  'lambda_l1': 0.0030357456951831487,
  'lambda_l2': 2.8619045146353634e-07,
  'num_leaves': 459,
  'feature_fraction': 0.5741898539418252,
  'bagging_fraction': 0.4954113493297779,
  'bagging_freq': 5,
  'min_child_samples': 98},
 0.23200185854242583)

In [16]:
trials_df = study.trials_dataframe()

In [18]:
study.optimize(objective, n_trials=50)

C:\Users\user\anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\user\anaconda3\lib\site-packages\lightgbm\engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[3772]	training's rmse: 0.000465085	training's RMSPE: 0.214771	valid_1's rmse: 0.00139956	valid_1's RMSPE: 0.232775
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[4135]	training's rmse: 0.000463787	training's RMSPE: 0.213277	valid_1's rmse: 0.00129215	valid_1's RMSPE: 0.229956
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[5000]	training's rmse: 0.00045673	training's RMSPE: 0.210533	valid_1's rmse: 0.00132448	valid_1's RMSPE: 0.227618
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[3757]	training's rmse: 0.000460858	training's RMSPE: 0.213555	valid_1's rmse: 0.0014885	valid_1's RMSPE: 0.242659
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[2541]	training's rmse: 0.000481226	training'

[I 2021-09-27 16:17:39,189] Trial 32 finished with value: 0.2334651270367585 and parameters: {'objective': 'rmse', 'learning_rate': 0.001875532765993416, 'lambda_l1': 0.0001865437752816735, 'lambda_l2': 0.0001226870818560249, 'num_leaves': 266, 'feature_fraction': 0.16522641377610076, 'bagging_fraction': 0.363025081923331, 'bagging_freq': 4, 'min_child_samples': 90}. Best is trial 26 with value: 0.23243422762975208.
C:\Users\user\anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\user\anaconda3\lib\site-packages\lightgbm\engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[2377]	training's rmse: 0.000448528	training's RMSPE: 0.207101	valid_1's rmse: 0.00138137	valid_1's RMSPE: 0.231625
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[2414]	training's rmse: 0.000449635	training's RMSPE: 0.206758	valid_1's rmse: 0.00127997	valid_1's RMSPE: 0.228661
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[2499]	training's rmse: 0.000447899	training's RMSPE: 0.206481	valid_1's rmse: 0.00131576	valid_1's RMSPE: 0.227019
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1964]	training's rmse: 0.000451834	training's RMSPE: 0.20939	valid_1's rmse: 0.00147821	valid_1's RMSPE: 0.240648
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1405]	training's rmse: 0.000472175	training's RMSPE: 0.2

[I 2021-09-27 16:31:28,449] Trial 33 finished with value: 0.2323130855744817 and parameters: {'objective': 'rmse', 'learning_rate': 0.0032930527496364253, 'lambda_l1': 1.6928840709938868e-05, 'lambda_l2': 0.0017437733500160078, 'num_leaves': 342, 'feature_fraction': 0.3958711373637057, 'bagging_fraction': 0.3539404703428774, 'bagging_freq': 5, 'min_child_samples': 92}. Best is trial 33 with value: 0.2323130855744817.
C:\Users\user\anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\user\anaconda3\lib\site-packages\lightgbm\engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


...

[I 2021-09-27 21:08:37,192] Trial 79 finished with value: 0.23239797599384515 and parameters: {'objective': 'l2', 'learning_rate': 0.012584540355934433, 'lambda_l1': 0.8876597568712691, 'lambda_l2': 1.0406749171697886e-08, 'num_leaves': 370, 'feature_fraction': 0.5083245876976346, 'bagging_fraction': 0.43190610636414284, 'bagging_freq': 6, 'min_child_samples': 98}. Best is trial 64 with value: 0.23200185854242583.
C:\Users\user\anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\user\anaconda3\lib\site-packages\lightgbm\engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[300]	training's l2: 2.13855e-07	training's RMSPE: 0.213752	valid_1's l2: 1.95178e-06	valid_1's RMSPE: 0.233258
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[351]	training's l2: 2.09749e-07	training's RMSPE: 0.210737	valid_1's l2: 1.65915e-06	valid_1's RMSPE: 0.228613
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[437]	training's l2: 2.01152e-07	training's RMSPE: 0.206827	valid_1's l2: 1.73298e-06	valid_1's RMSPE: 0.227122
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[297]	training's l2: 2.10063e-07	training's RMSPE: 0.212494	valid_1's l2: 2.20987e-06	valid_1's RMSPE: 0.242811
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[258]	training's l2: 2.22324e-07	training's RMSPE: 0.216534	valid_1's l2

[I 2021-09-27 21:11:42,839] Trial 80 finished with value: 0.23312782451124267 and parameters: {'objective': 'l2', 'learning_rate': 0.017526363424108262, 'lambda_l1': 1.3545001932436003, 'lambda_l2': 1.1384951542666492e-08, 'num_leaves': 372, 'feature_fraction': 0.5144880259569564, 'bagging_fraction': 0.345775704856592, 'bagging_freq': 6, 'min_child_samples': 100}. Best is trial 64 with value: 0.23200185854242583.
C:\Users\user\anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\user\anaconda3\lib\site-packages\lightgbm\engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1476]	training's l2: 2.09595e-07	training's RMSPE: 0.21136	valid_1's l2: 1.92616e-06	valid_1's RMSPE: 0.232025
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1152]	training's l2: 2.18822e-07	training's RMSPE: 0.215107	valid_1's l2: 1.68117e-06	valid_1's RMSPE: 0.229513
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1986]	training's l2: 2.01349e-07	training's RMSPE: 0.206808	valid_1's l2: 1.72467e-06	valid_1's RMSPE: 0.226942
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1185]	training's l2: 2.12258e-07	training's RMSPE: 0.213463	valid_1's l2: 2.21018e-06	valid_1's RMSPE: 0.241156
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[856]	training's l2: 2.29663e-07	training's RMSPE: 0.22001	valid_1's 

[I 2021-09-27 21:19:29,872] Trial 81 finished with value: 0.2327215121456789 and parameters: {'objective': 'l2', 'learning_rate': 0.005183240387472584, 'lambda_l1': 1.9017907220263157, 'lambda_l2': 2.4304448217974702e-08, 'num_leaves': 238, 'feature_fraction': 0.4262218545688707, 'bagging_fraction': 0.44773377451969315, 'bagging_freq': 6, 'min_child_samples': 98}. Best is trial 64 with value: 0.23200185854242583.


In [19]:
trials_df = study.trials_dataframe()
trials_df

,number,value,datetime_start,datetime_complete,duration,params_bagging_fraction,params_bagging_freq,params_feature_fraction,params_lambda_l1,params_lambda_l2,params_learning_rate,params_min_child_samples,params_num_leaves,params_objective,state
0,0,0.236647,2021-09-27 12:47:54.229435,2021-09-27 12:49:09.752738,0 days 00:01:15.523303,0.423106,7,0.719469,7.181883e-05,1.651824e-02,0.071583,52,19,rmse,COMPLETE
1,1,0.236555,2021-09-27 12:49:09.755666,2021-09-27 12:51:32.512780,0 days 00:02:22.757114,0.398044,5,0.059678,4.424387e-03,8.267192e-06,0.040701,54,336,l2,COMPLETE
2,2,0.233603,2021-09-27 12:51:32.514733,2021-09-27 12:54:00.838436,0 days 00:02:28.323703,0.849432,2,0.634401,5.669416e-08,2.576153e-02,0.044585,97,49,l2,COMPLETE
3,3,0.233931,2021-09-27 12:54:00.841363,2021-09-27 12:55:49.988045,0 days 00:01:49.146682,0.630976,3,0.293714,7.807136e-03,2.313198e-05,0.042304,74,292,rmse,COMPLETE
4,4,0.235246,2021-09-27 12:55:49.989997,2021-09-27 13:05:29.688989,0 days 00:09:39.698992,0.426351,6,0.312261,3.674165e-07,2.436353e-07,0.003770,35,148,rmse,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,77,0.232793,2021-09-27 20:54:13.961310,2021-09-27 20:58:22.129716,0 days 00:04:08.168406,0.606234,4,0.466853,6.629171e-03,4.487573e-07,0.016085,96,395,l2,COMPLETE
78,78,0.232402,2021-09-27 20:58:22.131667,2021-09-27 21:03:44.360915,0 days 00:05:22.229248,0.444933,6,0.513522,5.830156e-02,1.294632e-08,0.012036,99,463,l2,COMPLETE
79,79,0.232398,2021-09-27 21:03:44.362866,2021-09-27 21:08:37.192295,0 days 00:04:52.829429,0.431906,6,0.508325,8.876598e-01,1.040675e-08,0.012585,98,370,l2,COMPLETE
80,80,0.233128,2021-09-27 21:08:37.193271,2021-09-27 21:11:42.838355,0 days 00:03:05.645084,0.345776,6,0.514488,1.354500e+00,1.138495e-08,0.017526,100,372,l2,COMPLETE


In [57]:
trials_df.to_csv('lgbm_optuna_trials.csv', index=None)

In [48]:
trials_df = pd.read_csv('lgbm_optuna_trials.csv')

In [20]:
trials_df.sort_values('value', inplace=True)
trials_df.reset_index(inplace=True, drop=True)

In [21]:
trials_df.sort_values('value', inplace=True)
trials_df.reset_index(drop=True, inplace=True)
trials_df.head(7)

,number,value,datetime_start,datetime_complete,duration,params_bagging_fraction,params_bagging_freq,params_feature_fraction,params_lambda_l1,params_lambda_l2,params_learning_rate,params_min_child_samples,params_num_leaves,params_objective,state
0,64,0.232002,2021-09-27 19:25:12.469758,2021-09-27 19:28:58.168228,0 days 00:03:45.698470,0.495411,5,0.574190,0.003036,2.861905e-07,0.016879,98,459,l2,COMPLETE
1,33,0.232313,2021-09-27 16:17:39.190358,2021-09-27 16:31:28.448329,0 days 00:13:49.257971,0.353940,5,0.395871,0.000017,1.743773e-03,0.003293,92,342,rmse,COMPLETE
2,59,0.232320,2021-09-27 18:52:11.190553,2021-09-27 19:02:34.577337,0 days 00:10:23.386784,0.501925,6,0.602758,0.003038,2.942183e-06,0.005448,87,456,l2,COMPLETE
3,65,0.232323,2021-09-27 19:28:58.169203,2021-09-27 19:34:36.066030,0 days 00:05:37.896827,0.577379,5,0.475423,0.003531,6.617214e-08,0.012264,97,466,l2,COMPLETE
4,79,0.232398,2021-09-27 21:03:44.362866,2021-09-27 21:08:37.192295,0 days 00:04:52.829429,0.431906,6,0.508325,0.887660,1.040675e-08,0.012585,98,370,l2,COMPLETE
5,78,0.232402,2021-09-27 20:58:22.131667,2021-09-27 21:03:44.360915,0 days 00:05:22.229248,0.444933,6,0.513522,0.058302,1.294632e-08,0.012036,99,463,l2,COMPLETE
6,48,0.232410,2021-09-27 18:22:56.891778,2021-09-27 18:25:24.400462,0 days 00:02:27.508684,0.393896,6,0.740127,0.003974,7.049584e-06,0.025441,95,351,rmse,COMPLETE


In [ ]:
trials_df.iloc[1]

In [73]:
params = {
    'objective': 'mape',
    'metric': 'mape',
    'verbose': -1,
    'learning_rate': 0.017803626326238234,
    'lambda_l1': 0.0008363327284331296,
    'lambda_l2': 1.398098550350506e-07,
    'num_leaves': 443,
    'feature_fraction': 0.25837130684180626,
    'bagging_fraction': 0.946823931635175,
    'bagging_freq': 1,
    'min_child_samples': 13,
    'num_iterations': 10000,
    'early_stopping_round': 250,
}

ys = []
preds = []
models = []
# kfold
kfd = StratifiedKFold(5)

for i, (trn_idx,
        val_idx) in enumerate(kfd.split(df, y=df.stock_id, groups=df.time_id)):
    x_trn = x_data.iloc[trn_idx]
    y_trn = y_data.iloc[trn_idx]

    x_val = x_data.iloc[val_idx]
    y_val = y_data.iloc[val_idx]

    oof_pred, model, score = fit_lgbm(x_trn,
                                      y_trn,
                                      x_val,
                                      y_val,
                                      params,
                                      verbose=-1)
    models.append(model)
    ys.append(y_val)
    preds.append(oof_pred)
    print(f'fold{i} score: {score:.6f}')
ys = np.concatenate(ys, axis=0)
preds = np.concatenate(preds, axis=0)
oof_score = rmspe(preds, ys)
oof_score

C:\Users\user\anaconda3\lib\site-packages\lightgbm\engine.py:151: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\user\anaconda3\lib\site-packages\lightgbm\engine.py:156: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] feature_fraction is set=0.25837130684180626, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.25837130684180626
[LightGBM] [Warning] lambda_l1 is set=0.0008363327284331296, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0008363327284331296
[LightGBM] [Warning] bagging_fraction is set=0.946823931635175, subsample=1.0 will be ignored. Current value: bagging_fraction=0.946823931635175
[LightGBM] [Warning] lambda_l2 is set=1.398098550350506e-07, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.398098550350506e-07
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
Training until validation scores don't improve for 250 rounds
Early stopping, best iteration is:
[6753]	valid_0's mape: 0.00071484
fold0 score: 0.260299
[LightGBM] [Warning] feature_fraction is set=0.25837130684180626, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.25837130684180626
[Li

0.25720646039729733

In [77]:
save_pickle(models, 'cv2572064_kf_col53_win150_300_450.pkl')

In [76]:
# pickle
def save_pickle(dic, save_path):
    import pickle
    with open(save_path, 'wb') as f:
        # with gzip.open(save_path, 'wb') as f:
        pickle.dump(dic, f)


def load_pickle(load_path):
    import pickle
    with open(load_path, 'rb') as f:
        # with gzip.open(load_path, 'rb') as f:
        dic = pickle.load(f)
    return dic